In [1]:
from FairFedAvg import *

In [2]:
train(logReg(num_features=NUM_FEATURES, num_classes=2), 
      option = "FairBatch", optimizer = 'sgd', learning_rate = 0.01, 
      num_rounds = 6, local_epochs = 10, alpha = 0.01)

  0%|          | 0/6 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 90.750046
| Global Round : 0 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 53.202324
| Global Round : 0 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 64.683060
| Global Round : 0 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 57.928345
| Global Round : 0 | Local Epoch : 2 | [0/11188 (0%)]	Batch Loss: 47.527397
| Global Round : 0 | Local Epoch : 2 | [6400/11188 (57%)]	Batch Loss: 47.732994
| Global Round : 0 | Local Epoch : 3 | [0/11188 (0%)]	Batch Loss: 60.970001
| Global Round : 0 | Local Epoch : 3 | [6400/11188 (57%)]	Batch Loss: 41.861450
| Global Round : 0 | Local Epoch : 4 | [0/11188 (0%)]	Batch Loss: 65.220703
| Global Round : 0 | Local Epoch : 4 | [6400/11188 (57%)]	Batch Loss: 54.529682
| Global Round : 0 | Local Epoch : 5 | [0/11188 (0%)]	Batch Loss: 49.154243
| Global Round : 0 | Local Epoch : 5 | [6400/11188 (57%)]	Batch Loss: 46.927731
| Global Round : 0 | Local Epoc

 17%|█▋        | 1/6 [00:09<00:46,  9.36s/it]

Client 0: accuracy loss: 107.53 | fairness loss 0.36 | RD = 0.43 = |113/592-883/1421| 
Client 1: accuracy loss: 62.56 | fairness loss 0.50 | RD = 0.34 = |71/421-420/823| 
 
Avg Training Stats after 1 global rounds:
Training loss: 50.89 | Training accuracy: 76.17% | Training RD: 0.3990

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 66.182236
| Global Round : 1 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 32.836155
| Global Round : 1 | Local Epoch : 0 | [12800/11188 (114%)]	Batch Loss: 61.900139
| Global Round : 1 | Local Epoch : 0 | [19200/11188 (170%)]	Batch Loss: 56.955502
| Global Round : 1 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 40.113266
| Global Round : 1 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 45.543430
| Global Round : 1 | Local Epoch : 1 | [12800/11188 (114%)]	Batch Loss: 41.114567
| Global Round : 1 | Local Epoch : 1 | [19200/11188 (170%)]	Batch Loss: 40.681561
| Global Round : 1 | Local Epoch : 2 | [0

 33%|███▎      | 2/6 [00:23<00:43, 10.90s/it]

...

 83%|████████▎ | 5/6 [01:07<00:13, 13.36s/it]

Client 0: accuracy loss: 104.73 | fairness loss 0.46 | RD = 0.01 = |234/592-578/1421| 
Client 1: accuracy loss: 63.19 | fairness loss 0.51 | RD = 0.01 = |163/421-310/823| 
 
Avg Training Stats after 5 global rounds:
Training loss: 46.18 | Training accuracy: 77.45% | Training RD: 0.0038

 | Global Training Round : 6 |

| Global Round : 5 | Local Epoch : 0 | [0/11188 (0%)]	Batch Loss: 41.464336
| Global Round : 5 | Local Epoch : 0 | [6400/11188 (57%)]	Batch Loss: 39.428158
| Global Round : 5 | Local Epoch : 0 | [12800/11188 (114%)]	Batch Loss: 45.352737
| Global Round : 5 | Local Epoch : 0 | [19200/11188 (170%)]	Batch Loss: 53.230740
| Global Round : 5 | Local Epoch : 1 | [0/11188 (0%)]	Batch Loss: 37.346775
| Global Round : 5 | Local Epoch : 1 | [6400/11188 (57%)]	Batch Loss: 47.197998
| Global Round : 5 | Local Epoch : 1 | [12800/11188 (114%)]	Batch Loss: 43.269321
| Global Round : 5 | Local Epoch : 1 | [19200/11188 (170%)]	Batch Loss: 52.462257
| Global Round : 5 | Local Epoch : 2 | [

100%|██████████| 6/6 [01:22<00:00, 13.72s/it]

Client 0: accuracy loss: 103.69 | fairness loss 0.47 | RD = 0.01 = |228/592-530/1421| 
Client 1: accuracy loss: 62.71 | fairness loss 0.53 | RD = 0.03 = |155/421-280/823| 
 
Avg Training Stats after 6 global rounds:
Training loss: 45.72 | Training accuracy: 78.35% | Training RD: 0.0171


 
 Results after 6 global rounds of training:
|---- Avg Train Accuracy: 78.35%
|---- Test Accuracy: 80.42%
|---- Test RD: 0.0178

 Total Run Time: 82.7609 sec
